In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV

from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
SEED_VALUE = 42
LEARNING_RATE = 0.01
num_cores = multiprocessing.cpu_count()
num_cores

8

In [3]:
X_train = pd.read_csv("./Dataset/train_final.csv")
X_train.drop('Unnamed: 0', axis = 1, inplace=True)
X_train.head()

,Count,Open,Volume,Asset_ID,Target,Range_Close_Open,Range_High_Low
0,-0.420178,-0.231602,-0.163735,0,0.000148,0.009311,-0.106561
1,-0.406696,-0.231558,-0.163475,0,0.000393,0.009823,-0.106239
2,-0.412850,-0.231527,-0.163609,0,0.000549,0.009775,-0.106443
3,-0.411005,-0.231471,-0.163602,0,0.000006,0.010801,-0.105861
4,-0.389464,-0.230984,-0.162833,0,0.005618,0.011829,-0.102587


In [4]:
y_train = X_train['Target']
X_train.drop(["Target"], axis = 1, inplace=True)
X_train.head()

,Count,Open,Volume,Asset_ID,Range_Close_Open,Range_High_Low
0,-0.420178,-0.231602,-0.163735,0,0.009311,-0.106561
1,-0.406696,-0.231558,-0.163475,0,0.009823,-0.106239
2,-0.412850,-0.231527,-0.163609,0,0.009775,-0.106443
3,-0.411005,-0.231471,-0.163602,0,0.010801,-0.105861
4,-0.389464,-0.230984,-0.162833,0,0.011829,-0.102587


In [ ]:
XGBRegressor().get_params()

In [ ]:
# Tune the parameters using Grid search
param_grid = {
    "max_depth": list(np.arange(1, 100, step=24)),
    "n_estimators": list(np.arange(1, 500, step=10)),
    "max_leaves": ['0', '1', '2', '3']
}

model_xgb = XGBRegressor(n_jobs = num_cores - 1)

grid_search = GridSearchCV(model_xgb, param_grid, cv = 5, verbose = 2, scoring='neg_mean_squared_error', n_jobs = num_cores - 1)

In [ ]:
%%time
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
model_lgb = LGBMRegressor()
model_lgb.get_params()

In [ ]:
param_grid = {
    "max_depth": list(np.arange(1, 100, step=24)),
    "n_estimators": list(np.arange(1, 500, step=10)),
    "max_leaves": ['0', '1', '2', '3']
}


grid_search = GridSearchCV(model_lgb, param_grid, cv = 5, verbose = 2, scoring='neg_mean_squared_error', n_jobs = num_cores - 1)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [5]:
LassoCV().get_params()

{'alphas': None,
 'copy_X': True,
 'cv': None,
 'eps': 0.001,
 'fit_intercept': True,
 'max_iter': 1000,
 'n_alphas': 100,
 'n_jobs': None,
 'normalize': 'deprecated',
 'positive': False,
 'precompute': 'auto',
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'verbose': False}

In [13]:
param_grid = {
    "alphas": list(np.arange(0, 1, step=0.1)),
}

model_lasso = LassoCV(cv = 5)

grid_search = GridSearchCV(model_lasso, param_grid, cv = 5, verbose = 2, scoring='neg_mean_squared_error', n_jobs = num_cores - 1)

In [15]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/sameep/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sameep/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sameep/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py", line 1631, in fit
    alphas = np.tile(np.sort(alphas)[::-1], (n_l1_ratio, 1))
  File "<__array_function__ internals>", line 180, in sort
  File "/home/sameep/.local/lib/python3.8/site-packages/nump

AxisError: axis -1 is out of bounds for array of dimension 0